In [14]:
import os
from utils.peep import peep
from dotenv import load_dotenv
from langchain.document_loaders import PyMuPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

loader = PyMuPDFLoader("docs/coercive.pdf")
docs = loader.load()

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", 
                             api_key=os.getenv("GOOGLE_API_KEY"), 
                             max_tokens=1048576)

In [12]:
docs

[Document(metadata={'source': 'docs/coercive.pdf', 'file_path': 'docs/coercive.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '3B2 Total Publishing System 7.51n/W', 'producer': 'Apogee Create Series3 v1.0', 'creationDate': 'D:20081207084233Z', 'modDate': "D:20160918203010-07'00'", 'trapped': ''}, page_content='Can we justify eliminating coercive measures in\npsychiatry?\nE J D Prinsen,1 J J M van Delden2\n1 Zwolse Poort, Institute for\nMental Health West-Overijssel,\nRaalte, The Netherlands; 2 Julius\nCenter for Health Sciences and\nPrimary Care, University Medical\nCenter, Utrecht, The\nNetherlands\nCorrespondence to:\nDr E J D Prinsen, Zwolse Poort,\nInstitute for Mental Health\nWest-Overijssel,\nKnapenveldsweg 1/P.O. Box\n110, 8102 RR/8000 AC Raalte,\nThe Netherlands; e.prinsen@\nzwolsepoort.nl\nReceived 20 August 2007\nRevised 25 February 2008\nAccepted 28 April 2008\nABSTRACT\nThe practice of coercive me

In [17]:
total_content = "\n\n".join([doc.page_content for doc in docs])


toc_prompt = ChatPromptTemplate.from_messages([
    ("system", """Create a table of contents from the provided text. The full text is divided into several sections, with each section's first line containing a section heading or subheading. The font size of the (sub)headings determines the hierarchy of the sections. After determining the importance of each section based on the hierarchy, extract not more than 10 major sections and create a corresponding table of contents. Ignore references, footnotes, and other non-content text.
    """),
    ("user", "{text}"),
])

toc_chain = toc_prompt | llm

toc = toc_chain.invoke({"text": total_content}).content

toc

'## Table of Contents\n\n**1. Introduction**\n   - The practice of coercive measures in psychiatry is controversial.\n   - Ethical, legal, and clinical considerations are complex.\n   - This paper discusses the conflict between autonomy and beneficence/non-maleficence, human dignity, patient experiences, and the effects of coercive measures.\n\n**2. Background**\n   - Definition and prevalence of seclusion.\n   - Patient-related and non-patient-related influences on the use of seclusion.\n   - The decision to use seclusion depends on complex factors, including patient behavior, caregiver characteristics, and institutional culture.\n\n**3. Ethical Issues**\n   - **Autonomy:**\n     - The development of patient autonomy in healthcare.\n     - Autonomy is not the only moral consideration.\n     - Autonomy is relational and can be developed through interactions with others.\n     - Dependency is a fundamental moral principle.\n     - Coercive measures can be necessary to foster or regain a

In [35]:
summary_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """The provided text is composed of several sections and sub-sections structured by the following table of contents.
     {toc}
     Analyze and generate comprehensive thoughts about the content of each sub-section structured by the table of contents one by one consecutively.
     - No subsection should be left out in the output.
     - The thoughts for each sub-section should be written with a thoughtful and in-depth approach to uncover valuable insights.
     - The thoughts for each sub-section should be written in a single paragraph, do not use bullet points, and it should be at least 1500 characters long but not more than 2000 characters.
     - Use **bold** formatting to emphasize key points.
    """),
    ("user", "{text}"),
])

summary_template = summary_prompt_template.partial(toc=toc)

summary_chain = summary_template | llm

summary = summary_chain.invoke({"text": total_content}).content


In [36]:
from IPython.display import Markdown

Markdown(summary)

## 1. Introduction

This paper aims to delve into the complex and controversial issue of coercive measures in psychiatry, specifically seclusion. The authors acknowledge the ethical, legal, and clinical complexities surrounding this practice, particularly when considering the temporary nature of mental incapacity and the potential for coercive measures to restore autonomy. The paper examines the tension between autonomy and beneficence/non-maleficence, human dignity, patient experiences, and the effects of coercive measures. Ultimately, the authors argue that while respect for autonomy and human dignity are crucial considerations, they are not sufficient reasons to completely reject coercive measures. 

## 2. Background

The paper establishes a clear definition of seclusion as the practice of isolating a patient in a room for safety and control of behavior. The prevalence of seclusion remains unclear, with significant variations across countries and institutions, making comparisons difficult.  The decision to seclude a patient is influenced by a complex interplay of factors, including patient behavior, caregiver characteristics, and institutional culture. This complexity highlights the need for a nuanced discussion about the justification of a complete non-seclusion approach. 

## 3. Ethical Issues

The authors meticulously analyze the ethical implications of seclusion, focusing on autonomy and human dignity.

**Autonomy**: The authors argue that while the trend towards patient autonomy in healthcare is commendable, autonomy is not an absolute moral principle and cannot override all other considerations.  Autonomy is relational and can be developed through interactions with others. Furthermore, dependency is a fundamental moral principle that can coexist with autonomy. Coercive measures, like seclusion, may be necessary to foster or regain autonomy in cases of self-destructive behavior.

**Human Dignity**: The authors explore two conceptions of human dignity: inherent and individualistic. While seclusion violates inherent dignity, the violation of individualistic dignity is more nuanced. Seclusion can intrude on a patient's private sphere and affect their identity, potentially violating their dignity. However, respect for human dignity might focus on how seclusion is used rather than whether it should be used at all.  The authors posit that patients may have already experienced a loss of dignity due to their illness and that seclusion could be necessary to regain self-respect and dignity.

**Experiences of Patients**: Patients report both negative and positive experiences with seclusion. Negative experiences are often linked to a lack of information and education about the procedure. However, patients may also recognize the potential benefits of seclusion, particularly in calming and providing a respite from overwhelming stimulation.  This suggests that the way seclusion is used, including communication and education, significantly influences patient experience.

**Effects of Seclusion; Promoting Wellbeing**: The authors highlight the lack of evidence to support the effectiveness of seclusion, despite its intended purpose of protecting patients and creating a therapeutic environment. While some clinicians report positive effects, the absence of controlled trials leaves the efficacy of seclusion in question.  The authors also cite evidence of negative effects of seclusion, including physical and psychological harm. This lack of evidence for positive effects and the presence of documented negative effects strongly advocates for a non-seclusion approach.

## 4. Discussion

The authors emphasize the ambiguity of ethical concepts like autonomy and human dignity, acknowledging that these concepts can be used to both support and reject a non-seclusion approach. While reduction programs are encouraged, the authors highlight that these programs do not eliminate coercive measures entirely. The lack of clear definitions, data, and controlled trials comparing different coercive measures further complicates the issue.  The authors raise the possibility that forced drug treatment, often seen as an alternative to seclusion, might be even more harmful due to its invasiveness and potential for violating autonomy. 

## 5. Conclusion

The authors conclude that autonomy and human dignity are not sufficient reasons to eliminate seclusion entirely.  The lack of controlled trials on the effects of seclusion strongly argues against its use, even in extreme circumstances. The authors advocate for the use of seclusion only within the context of randomized trials to determine its effectiveness and compare it to other measures. In the absence of evidence for positive effects, the precautionary principle of "primum non nocere" (first, do no harm) should guide our actions.


In [32]:
summary

'## 1. Introduction\n\nThe paper delves into the complex issue of coercive measures in psychiatry, acknowledging the ethical, legal, and clinical challenges associated with their use. The authors highlight the inherent conflict between **autonomy** and **beneficence/non-maleficence** principles, emphasizing the need for a nuanced discussion that considers human dignity, patient experiences, and the potential effects of such measures. The introduction sets the stage for a critical examination of the ethical and practical considerations surrounding the use of coercive measures in mental health treatment. \n'